


# 프롬프트 생성 및 등장인물 변환기




In [ ]:
!pip install gliner
!pip install python-mecab-ko
!pip install gradio

In [ ]:
charList = ['C001',
 'C002',
 'C003',
 'C004',
 'C005',
 'C006',
 'C007',
 'C008',
 'C009',
 'C010',
 'C011',
 'C012',
 'C013',
 'C014',
 'C015',
 'C016',
 'C017',
 'C018',
 'C019',
 'C020',
 'C021',
 'C022',
 'C023',
 'C024',
 'C025',
 'C026',
 'C027',
 'C028',
 'C029',
 'C030',
 'C031',
 'C032',
 'C033',
 'C034',
 'C035',
 'C036',
 'C037',
 'C038',
 'C039',
 'C040',
 'C041',
 'C042',
 'C043',
 'C044',
 'C045',
 'C046',
 'C047',
 'C048',
 'C049',
 'C050',
 'C051',
 'C052',
 'C053',
 'C054',
 'C055',
 'C056',
 'C057',
 'C058',
 'C059',
 'C060',
 'C061',
 'C062',
 'C063',
 'C064',
 'C065',
 'C066',
 'E001',
 'E002',
 'E003',
 'E004',
 'E005',
 'E007',
 'E009',
 'E011',
 'E025']

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

#토큰들과 토크나이저 설정
Q_TKN = "<usr>"
A_TKN = "<sys>"
SENT = "<unused1>"
EOS = "</s>"
BOS = "</s>"

koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token=BOS,
    eos_token=EOS,
    unk_token="<unk>",
    pad_token="<pad>",
    mask_token="<unused0>",
)

#토크나이저에 새로운 토큰 추가
#등장인물들 새로운 토큰으로 추가하여 예외적으로 등장인물이 잘려서 토큰화되지 않도록 함

koGPT2_TOKENIZER.add_tokens(charList)


#Load your trained model
#resize_token_embeddings를 명시해줘야 제대로 작동
model_path = "/content/drive/MyDrive/deepdaivproject/stage_emotion_final/storymaker_model.pth" # Adjust the path to your saved model
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
model.resize_token_embeddings(len(koGPT2_TOKENIZER))


#저장된 커스텀 모델경로 복사
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint["model_state_dict"],strict=False)
model.eval()

#Interaction loop with the chatbot
end = False
with torch.no_grad():
    conversation_history = []
    num=0
    while not end:
        #if문은 처음 사용자가 문장을 입력했을 때
        if num==0:
          q = input("user > ").strip()
          if q == "quit":
              break
          #사용자가 감정+대사 또는 서술 순으로 작성하기 때문에 학습한 패턴대로 토큰을 넣어주기
          #위해서 문장을 쪼갬
          q1 = q[0:2]
          q2 = q[3:]
          #학습했던 토큰의 순서대로 넣어줌
          q = Q_TKN+q1+SENT+q2+SENT+A_TKN
          #print(q)
          gen_ids = model.generate(koGPT2_TOKENIZER.encode(q, return_tensors='pt'),
                      max_length=256,
                      pad_token_id=koGPT2_TOKENIZER.pad_token_id,
                      eos_token_id=koGPT2_TOKENIZER.eos_token_id,
                      bos_token_id=koGPT2_TOKENIZER.bos_token_id,
                      use_cache=True,top_p=0.75,temperature=0.6,repetition_penalty=1.5,do_sample=True)
          generated = koGPT2_TOKENIZER.decode(gen_ids[0],)
          #print(generated)

          #<sys>와 </s> 사이의 텍스트 추출
          result = re.search(r"<sys>(.*?)</s>", generated)
           # 결과 출력
          if result:
                generated = result.group(1).strip()
                #<sys>와 </s> 사이의 텍스트만 추출하고 공백 제거
          else:
                print("해당 패턴을 찾을 수 없습니다.")

          #print(generated)
          #1. 불필요한 태그 제거 (<unused1>, <sys>, </s> 등)
          #cleaned_text = re.sub(r"<unused1>|<sys>|</s>|<usr>|</s>", "",generated)


          #2. 불필요한 공백 제거할려고 했으나 공백이 있는 것이 토큰화에 유리하기 때문에 배제
          #cleaned_text = cleaned_text.replace(" ", "")
          #print(cleaned_text)

          #3 conversation_history에 추가
          #사용자가 처음 문장을 입력했을 때에는 이전 문장이 존재하지 않으므로 현재 문장을 추가만 해준다.
          conversation_history.append(generated)
          #conversation_history.append('안녕하세요')
          #print(conversation_history)


          print(generated)
          num+=1

        #사용자가 작성한 문장이 처음이 아닐 때
        else:
          q = input("user > ").strip()
          if q == "quit":
             break
          q1 = q[0:2]
          q2 = q[3:]
          #print(q1)
          #print(q2)
          #바로 이전 문장만 기억하게 할 것이기 때문에 pop 함수 사용해서 꺼내고 conversation_history list는 비어있도록 만듬
          q3 = conversation_history.pop()
          q3 = str(q3)
          q = Q_TKN+q1+SENT+q3+q2+SENT+A_TKN
          #print(q)
          gen_ids = model.generate(koGPT2_TOKENIZER.encode(q, return_tensors='pt'),
                      max_length=512,
                      pad_token_id=koGPT2_TOKENIZER.pad_token_id,
                      eos_token_id=koGPT2_TOKENIZER.eos_token_id,
                      bos_token_id=koGPT2_TOKENIZER.bos_token_id,
                      use_cache=True,top_p=0.85,temperature=0.6, repetition_penalty = 1.5 ,do_sample=True)

          generated = koGPT2_TOKENIZER.decode(gen_ids[0],)
          #print(generated)
          #<sys>와 </s> 사이의 텍스트 추출
          result = re.search(r"<sys>(.*)</s>", generated)

          #결과 출력
          if result:
                generated = result.group(1).strip()  # <sys>와 </s> 사이의 텍스트만 추출
          else:
                print("해당 패턴을 찾을 수 없습니다.")

          #<sys>와 </s> 사이의 텍스트만 추출하면 special token 존재하지 않기 때문에 special token 제거과정은 생략
          #generated = re.sub(r"<pad>", r"", generated)
          #generated = re.sub(r"<sys>",r"",generated)
          #generated = re.sub(r"<unused0>",r"",generated)
          #generated = re.sub(r"<unused1>",r"",generated)
          #generated = re.sub(r"</s>",r"",generated)
          a = "Bot > "
          print(a + generated)

          #서사구조 자르기
          #con_generated = generated[0:2]
          #서사구조 자른 문장 conversation_history에 저장
          conversation_history.append(q2+generated)
          #print(conversation_history)

In [ ]:
import pandas as pd

name_df = pd.read_csv('name.csv파일경로') #name.csv 파일 다운받고 경로 복사
name_df

In [ ]:
charNameList = name_df['char'].to_list()
nameList = name_df['이름'].to_list()

tup_list = list(zip(charNameList, nameList))

char_to_name = dict(tup_list)
name_to_char = {v: k for k, v in char_to_name.items()}

print(len(char_to_name))
print(len(name_to_char))

In [ ]:
from gliner import GLiNER

NERmodel = GLiNER.from_pretrained("taeminlee/gliner_ko")

In [ ]:
def model_input_char_transformer(input):
  tta_labels = ["PERSON"]
  entities = NERmodel.predict_entities(input, tta_labels)
  person_entities = [entity for entity in entities if entity['label'] == 'PERSON']

  print(person_entities)

  for idx, entity in enumerate(person_entities):
    for name in name_to_char.keys():
      if name in entity['text']:
        entity['text'] = entity['text'].replace(name, name_to_char[name])

        diff = len(name_to_char[name]) - len(name)
        for en in person_entities[idx+1:]:
          en['start'] += diff
          en['end'] += diff

        print(person_entities)
        break

  for entity in person_entities:
    input = input[:entity['start']] + entity['text'] + input[entity['end']:]

  return input

def model_output_char_transformer(input):
  for char in charList:
    input = input.replace(char, char_to_name[char])

  return input

# Gradio로 구현

In [ ]:
user_chars = input("사용할 등장인물의 이름을 적으세요(띄어쓰기로 구분) > ").strip()
user_chars_list = user_chars.split()

print(user_chars_list)

for index, user_char in enumerate(user_chars_list):
  if user_char in name_to_char.keys():
    temp_char = name_to_char[user_char] # E056 - 민수
    print(temp_char)
    temp_name = char_to_name[charList[index]] # 서연 - C003
    print(temp_name)

    name_to_char[user_char] = charList[index]
    name_to_char[temp_name] = temp_char

    char_to_name[charList[index]] = user_char
    char_to_name[temp_char] = temp_name

  else:
    temp_name = char_to_name[charList[index]] # 성희 - C002
    print(temp_name)

    del char_to_name[charList[index]]
    char_to_name[charList[index]] = user_char

    del name_to_char[temp_name]
    name_to_char[user_char] = charList[index]

In [ ]:
import gradio as gr

whole_story = ""
last_sent = ""  # 마지막 입력과 출력된 문장 저장
previous_story = ""  # 바로 전 문장을 제외한 스토리 저장
previous_input_output = ""  # 바로 전 입력과 출력된 문장 저장

def storymaker(input, emo, struct, regenerate=False):
    global whole_story
    global last_sent
    global previous_story
    global previous_input_output  # 바로 전 입력과 생성된 문장을 저장하는 변수

    if regenerate:
        # regenerate 시 바로 전에 생성된 문장과 입력만 사용
        story_context = previous_input_output + input  # 이전 입력/출력 + 새 입력
        whole_story = previous_story  # 이전 스토리로 복원
        whole_story += input  # 새로 입력한 문장을 whole_story에 추가
    else:
        if input == 'quit' or input == '가을이었다.':
            return '종료합니다.' + '\n' + whole_story

        previous_story = whole_story  # 현재까지의 스토리를 저장
        whole_story += input  # 새로운 입력을 whole_story에 반영

        # 이전 입력과 출력된 문장 저장
        previous_input_output = last_sent

        # 문장을 생성할 때 바로 이전의 입력과 생성된 출력만 사용
        story_context = last_sent + input
        input = model_input_char_transformer(input)  # 입력 변환

    # 입력 문장 + 감정 + 서사 구조 정보를 기반으로 문장 생성
    q = Q_TKN + emo+ SENT + struct + SENT + story_context + SENT + A_TKN

    gen_ids = model.generate(tokenizer.encode(q, return_tensors='pt'),
                        max_length=256,
                         temperature=0.3,
                         top_p=0.75,
                        # top_k=50,
                        do_sample=True,
                        repetition_penalty=2.0,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        use_cache=True)

    generated = tokenizer.decode(gen_ids[0])
    generated = generated.split(A_TKN)[1]
    generated = generated.split(EOS)[0]

    generated = model_output_char_transformer(generated)
    whole_story += generated + "\n"

    last_sent = input + generated  # 마지막 입력 문장과 생성된 문장 저장

    return whole_story, generated

def show_whole_story():
    return whole_story


with gr.Blocks() as demo:
    story_state = gr.State(whole_story)  # 스토리를 상태로 저장
    last_sent_state = gr.State(last_sent)  # 마지막 문장을 상태로 저장
    previous_story_state = gr.State(previous_story)  # 이전 스토리 상태 저장
    previous_input_output_state = gr.State(previous_input_output)  # 이전 입력/출력 저장

    chatbot = gr.Chatbot(label="AI와 소설 만들기")
    with gr.Row():
        input_text = gr.Textbox(label="입력", placeholder="가을이었다.")
    with gr.Row():
        struct_choice = gr.Radio(
            ["발단", "전개", "위기", "절정", "결말"], label="서사 구조 단계", info="다음 문장으로 나왔으면 하는 문장의 서사 구조 단계를 선택하세요."
        )
        emo_choice = gr.Radio(
            ["무덤덤", "동요", "관심", "슬픔", "분노", "행복"], label="감정", info="다음 문장으로 나왔으면 하는 문장의 감정을 선택하세요."
        )

    # 전체 스토리를 보여주는 텍스트 박스
    whole_story_output = gr.Textbox(label="전체 스토리", placeholder="지금까지의 이야기가 여기에 표시됩니다.", interactive=False)

    def generate_story(messages, input_text, emo_choice, struct_choice, regenerate, whole_story, last_sent, previous_story, previous_input_output):
        story, generated = storymaker(input_text, emo_choice, struct_choice, regenerate)

        # 다시 생성일 경우 이전 AI 응답 삭제
        if regenerate:
            # 마지막 AI 메시지를 삭제
            if len(messages) > 0:
                messages.pop(-1)  # 마지막 AI 메시지 제거
                messages.pop(-1)  # 마지막 사용자 입력 제거

        # 사용자 입력과 AI의 새 출력 추가
        messages.append((input_text,None))
        messages.append((None,generated))

        return messages, story, last_sent, previous_story, previous_input_output

    generate_button = gr.Button("생성")
    regenerate_button = gr.Button("다시 생성")
    whole_story_button = gr.Button('전체 스토리')

    generate_button.click(generate_story,
                          [chatbot, input_text, emo_choice, struct_choice, gr.State(False), story_state, last_sent_state, previous_story_state, previous_input_output_state],
                          [chatbot, story_state, last_sent_state, previous_story_state, previous_input_output_state])

    regenerate_button.click(generate_story,
                            [chatbot, input_text, emo_choice, struct_choice, gr.State(True), story_state, last_sent_state, previous_story_state, previous_input_output_state],
                            [chatbot, story_state, last_sent_state, previous_story_state, previous_input_output_state])

    # 전체 스토리 버튼 클릭 시 전체 스토리 출력
    whole_story_button.click(show_whole_story, [], whole_story_output)

demo.launch(share=True)